This notebook converts the csv files to pickle files for quick processing in python.
Prerequisites: all the csv files of the mimics-iii database is dumped into a folder

In [1]:
from dotenv import load_dotenv
load_dotenv()
from glob import glob
import pandas as pd
import os,sys
import pandas as pd
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)


import os

folder_containing_mimics_csv_files = os.environ.get("MIMICPATH")
print(folder_containing_mimics_csv_files)


/usr/lib/python3/dist-packages/pytz/__init__.py:31: SyntaxWarning: invalid escape sequence '\s'
  match = re.match("^#\s*version\s*([0-9a-z]*)\s*$", line)


/home/asusaiyah/data/tactics_storage/mimic-iii/


In [2]:
print(os.environ.get("MIMICSPATH"))

None


In [3]:

filelist = sorted(glob(f"{folder_containing_mimics_csv_files}/*.csv"), key=os.path.getsize)
print(filelist)
for file_path in filelist:
    os.makedirs("../data",exist_ok=True)
    op_file = os.path.join("../data/", os.path.basename(file_path).replace(".csv", ".pkl"))
    print(f"Processing {file_path}...")
    if os.path.exists(op_file):
        print(f"Skipping {file_path}, already processed.")
        continue
    if "ADMISSIONS.csv" in file_path:
        parse_cols =  ["ADMITTIME", "DISCHTIME"]
    else:
        parse_cols = None
        
    chunks = pd.read_csv(file_path, parse_dates=parse_cols, chunksize=10000)
    all_chunks = []
    for chunk in chunks:
        all_chunks.append(chunk)
    combined_df = pd.concat(all_chunks)
    if "HADM_ID" in combined_df.columns:
        combined_df = combined_df[~combined_df.HADM_ID.isna()]
        combined_df["HADM_ID"] = combined_df["HADM_ID"].astype(int)
    combined_df.to_pickle(op_file)


['/home/asusaiyah/data/tactics_storage/mimic-iii/CAREGIVERS.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/PATIENTS.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/SERVICES.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/PROCEDURES_ICD.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/ADMISSIONS.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/TRANSFERS.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/PROCEDUREEVENTS_MV.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/MICROBIOLOGYEVENTS.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/OUTPUTEVENTS.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/PRESCRIPTIONS.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/INPUTEVENTS_MV.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/LABEVENTS.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/INPUTEVENTS_CV.csv', '/home/asusaiyah/data/tactics_storage/mimic-iii/NOTEEVENTS.csv']
Processing /home/asusaiyah/data/tactics_storage/mimic-iii/CARE

In [4]:
# Display all columns in all the files


# Path to the folder containing pickle files
folder_path = "../data"
os.makedirs(folder_path,exist_ok=True)

# Iterate through all files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".pkl"):  # Check if the file is a pickle file
        file_path = os.path.join(folder_path, file_name)
        try:
            # Load the pickle file into a DataFrame
            df = pd.read_pickle(file_path)
            print(f"Columns in {file_name}:")
            print(df.columns.tolist())
            print()
        except Exception as e:
            print(f"Error loading {file_name}: {e}")


Columns in MICROBIOLOGYEVENTS.pkl:
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'SPEC_ITEMID', 'SPEC_TYPE_DESC', 'ORG_ITEMID', 'ORG_NAME', 'ISOLATE_NUM', 'AB_ITEMID', 'AB_NAME', 'DILUTION_TEXT', 'DILUTION_COMPARISON', 'DILUTION_VALUE', 'INTERPRETATION']

Columns in PRESCRIPTIONS.pkl:
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE', 'ENDDATE', 'DRUG_TYPE', 'DRUG', 'DRUG_NAME_POE', 'DRUG_NAME_GENERIC', 'FORMULARY_DRUG_CD', 'GSN', 'NDC', 'PROD_STRENGTH', 'DOSE_VAL_RX', 'DOSE_UNIT_RX', 'FORM_VAL_DISP', 'FORM_UNIT_DISP', 'ROUTE']

Columns in ADMISSIONS.pkl:
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME', 'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA']

Columns in INPUTEVENTS_CV.pkl:
['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'AMOUNT', 'AM